# SA score

Take all generated molecules from guacamol benchmark => run through SA score

# Test set rediscovery/Similarity search

1. Take L1000 test set's gene expressions 
2. For each pair of tumour and control gene expressions => compute a difference vector 
3. Use euclidean distance and cosine distance between difference vector and every other difference vector in train set
4. Choose the one with smallest distance (2 candidates per test set molecule)
5. Generate 10 * num_possible_difference_vectors molecules in total for the set of possible pairs in test set using gene expression (Model will get 10 tries per difference vector)
6. Compute max structural similarity between the generated molecules and their respective test set molecules 
7. Plot the distribution of max structural similarity for the similarity search and the max structural similarity for generated molecules (probability density plot)

### train validation test split must be such that test set doesn't have molecules in the train set: this is so that during similarity search, we don't want to get the exact same molecule


# Protein target (as per original paper)
1. Given a protein target, we want to find the known molecules that inhibit it AND also appear in the l1000 dataset (so that we have gene expressions for them) 
2. Given these gene expressions we can similarity search within the training set
3. Then we can compare these found molecules with ALL the known inhibitors for that protein

### train validation test split will be based on which of the molecules are in the Excape database => all those belonging to the protein targets will have to be in the test set and not in the train set